In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import expected_returns, risk_models, BlackLittermanModel
import yfinance as yf

In [2]:
data = {}
start_date = '2020-01-01'
end_date = '2020-12-31'

In [6]:
data['VTI'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-06-15,55.424999,56.005001,55.174999,55.665001,38.037083,1067400
2001-06-18,55.814999,55.915001,55.310001,55.310001,37.794502,282600
2001-06-19,56.150002,56.165001,55.235001,55.465000,37.900406,1777600
2001-06-20,55.375000,56.075001,55.375000,56.049999,38.300152,476000
2001-06-21,56.000000,56.724998,55.875000,56.500000,38.607647,240400


In [3]:
data['VTI'] = yf.download('VTI', period='max')
data['VTV'] = yf.download('VTV', period='max')
data['VOE'] = yf.download('VOE', period='max')
data['VBR'] = yf.download('VBR', period='max')
data['GBIL'] = yf.download('GBIL', period='max')
data['JPST'] = yf.download('JPST', period='max')
data['VNQ'] = yf.download('VNQ', period='max')
data['VPU'] = yf.download('VPU', period='max')
#data['BITO'] = yf.download('BITO', period='max')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


for ticker in data:
    data[ticker]['percentageChange'] = data[ticker]['Close'].pct_change()*10
    data[ticker]['percentageChange'] = data[ticker]['percentageChange'].fillna(0)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import time #helper libraries

In [5]:
scalar = MinMaxScaler(feature_range=(0, 1))

In [6]:
data['VTI'] = scalar.fit_transform(data['VTI'].dropna().values.reshape(-1, 1))
data['VTV'] = scalar.fit_transform(data['VTV'].dropna().values.reshape(-1, 1))
data['VOE'] = scalar.fit_transform(data['VOE'].dropna().values.reshape(-1, 1))
data['VBR'] = scalar.fit_transform(data['VBR'].dropna().values.reshape(-1, 1))
data['GBIL'] = scalar.fit_transform(data['GBIL'].dropna().values.reshape(-1, 1))
data['JPST'] = scalar.fit_transform(data['JPST'].dropna().values.reshape(-1, 1))
data['VNQ'] = scalar.fit_transform(data['VNQ'].dropna().values.reshape(-1, 1))
data['VPU'] = scalar.fit_transform(data['VPU'].dropna().values.reshape(-1, 1))
#data['BITO'] = scalar.fit_transform(data['BITO'].dropna().values.reshape(-1, 1))

In [7]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [8]:
trainingData = {}
testData = {}

In [9]:
for ticker in data:
    trainingData[ticker], testData[ticker] = train_test_split(data[ticker], test_size=0.2)

In [10]:
trainX, trainY = {}, {}
testX, testY = {}, {}
timeStep = 100
for ticker in data:
    trainX[ticker], trainY[ticker] = create_dataset(trainingData[ticker], timeStep)
    testX[ticker], testY[ticker] = create_dataset(testData[ticker], timeStep)

In [11]:
trainX['VTI'].shape

(25214, 100)

In [12]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(trainX['VTI'].shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [14]:
for ticker in data:
    model.fit(trainX[ticker], trainY[ticker], epochs=1, batch_size=32, verbose=1, validation_data=(testX[ticker], testY[ticker]))
    model.save(ticker + '.h5')
    model.save_weights(ticker + '_weights.h5')

690/690 [==============================] - 65s 95ms/step - loss: 2.0470e-04 - root_mean_squared_error: 0.0143 - val_loss: 1.3730e-04 - val_root_mean_squared_error: 0.0117


In [15]:
viewDict = {}

In [24]:
models = {}
for ticker in data:
    print(ticker)
    models[ticker] = load_model(ticker + '.h5')
    viewDict[ticker] = model.predict(testX[ticker])

VTI
VTV
VOE
VBR
GBIL
JPST
VNQ
VPU


In [23]:
trainX['VTI'].shape

(25214, 100)

In [25]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.black_litterman import BlackLittermanModel

In [27]:
viewDict['VTI']

array([[0.00412347],
       [0.00412464],
       [0.00412393],
       ...,
       [0.00405081],
       [0.00405091],
       [0.00405097]], dtype=float32)

In [56]:
bl = BlackLittermanModel(mu=expected_returns.mean_historical_return(data),absolute_views=viewDict,sigma=risk_models.sample_cov(data),R=None)

ValueError: If using all scalar values, you must pass an index

In [7]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

viewdict = {"AAPL": 0.20, "BBY": -0.30, "BAC": 0, "SBUX": -0.2, "T": 0.15}
bl = BlackLittermanModel(cov_matrix, absolute_views=viewdict)

rets = bl.bl_returns()
ef = EfficientFrontier(rets, cov_matrix)

# OR use return-implied weights
delta = black_litterman.market_implied_risk_aversion(market_prices)
bl.bl_weights(delta)
weights = bl.clean_weights()

NameError: name 'cov_matrix' is not defined